In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from sklearn import preprocessing
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Describing Data

This data set includes customers who have paid off their loans, who have been past due and put into collection without paying back their loan and interests, and who have paid off only after they were put in collection. The financial product is a bullet loan that customers should pay off all of their loan debt in just one time by the end of the term, instead of an installment schedule. Of course, they could pay off earlier than their pay schedule.

* Loan_id: A unique loan number assigned to each loan customers

* Loan_status: Whether a loan is paid off, in collection, new customer yet to pay off, or paid off after the collection efforts
 
* Principal: Basic principal loan amount at the origination
 
* terms: Can be weekly (7 days), biweekly, and monthly payoff schedule
 
* Effective_date: When the loan got originated and took effects

* Due_date: Since it’s one-time payoff schedule, each loan has one single due date

* Paidoff_time: The actual time a customer pays off the loan

* Pastdue_days: How many days a loan has been past due

* Age, education, gender A customer’s basic demographic information

## Load and Check Data

In [ ]:
df=pd.read_csv("/kaggle/input/loandata/Loan payments data.csv")

In [ ]:
df.info

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.isnull().sum()

* paid_off_time and past_due_days have null values.

In [ ]:
{column:len(df[column].unique()) for column in df.columns}

In [ ]:
list=["loan_status","education","Gender"]
for i in list:
    print(df[i].unique())

# Filling Missing Value: paid_off_time

In [ ]:
g=sns.factorplot(x="loan_status",y="past_due_days",data=df,kind="box")
plt.show()

In [ ]:
print(df.loan_status.unique())

fig = plt.figure(figsize=(5,5))
ax = sns.countplot(df.loan_status)
ax.set_title("Count of Loan Status")
for p in ax.patches:
    ax.annotate(str(format(int(p.get_height()), ',d')), (p.get_x(), p.get_height()*1.01))
plt.show()

* Put both of plot together!
* There are 300 missing values in past_due_days
* Paidoff category have not any past_due_days
* The count of paidoff category is 300.
* So we can say that there are not missing values in past_due_days actually.

In [ ]:
df['past_due_days'] = df['past_due_days'].fillna(0)

In [ ]:
df.head()

In [ ]:
df[df["paid_off_time"].isnull()].head(20)

In [ ]:
df[df["paid_off_time"].isnull()].tail(21)

* The paid_off_time has 100 missing values.
* The count of COLLECTION is 100.
* we can see that from data, missing values of paid_off_time belongs to COLLECTION

In [ ]:
df['paid_off_time'] = df['paid_off_time'].fillna(-1)

# Visualization

In [ ]:
sns.pairplot(df)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 5))
sns.boxplot(x='loan_status', y='Principal', data=df, hue='loan_status', ax=axs[0])
sns.distplot(df.Principal)
plt.show();

In [ ]:
#past_due_days-Gender
g=sns.barplot(x="Gender",y="past_due_days",data=df)

In [ ]:
#past_due_days-education-Gender
g=sns.barplot(x="education",y="past_due_days",data=df,hue="Gender")

In [ ]:
#loan_status-past_due_days
g=sns.violinplot(x="loan_status",y="past_due_days",data=df)

# Pre-processing/EDA

In [ ]:
#Firstly,drop loan_id. loan_id is unnecessary because it does not effect anything.
df.drop(["Loan_ID"],inplace=True,axis=1)

In [ ]:
df.head()

In [ ]:
df['effective_date'] = pd.to_datetime(df['effective_date'])
df['due_date'] = pd.to_datetime(df['due_date'])
df['paid_off_time'] = pd.to_datetime(df['paid_off_time']).dt.date

In [ ]:
df['paid_off_time'] = pd.to_datetime(df['paid_off_time'])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.drop(["effective_date"],inplace=True,axis=1)
df.drop(["due_date"],inplace=True,axis=1)
df.drop(["paid_off_time"],inplace=True,axis=1)


In [ ]:
#To use models we should convert categorical values.
#I will seperate numerical and categorical variable from each other
categorical=[]
numerical=[]
for i,c in enumerate(df.dtypes):
    if c == object:
        categorical.append(df.iloc[:, i])
    else :
        numerical.append(df.iloc[:, i])


In [ ]:
categorical = pd.DataFrame(categorical).transpose()
numerical = pd.DataFrame(numerical).transpose()

In [ ]:
categorical.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder  
le = LabelEncoder()
categorical.head()

In [ ]:
target_values = {'PAIDOFF': 0 , 'COLLECTION' : 1,'COLLECTION_PAIDOFF' : 2}

target = categorical['loan_status']
categorical.drop('loan_status', axis=1, inplace=True)

target = target.map(target_values)

In [ ]:
for i in categorical:
    categorical[i] = le.fit_transform(categorical[i])


In [ ]:
target.head()

In [ ]:
categorical.head()

In [ ]:
df = pd.concat([categorical, numerical, target], axis=1)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
X = pd.concat([categorical, numerical], axis=1)
y = target 

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
corr=df.corr()
top_corr_features=corr.index
plt.figure(figsize=(10,10))

g=sns.heatmap(df[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
df.shape

In [ ]:
y=y.values.reshape(500,1)

In [ ]:
y.shape

# Modelling

## K-Nearest Neighbour

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
#find best k value for knn algorithm
from sklearn.neighbors import KNeighborsClassifier

score_list=[]
for i in range(2,15):
    knn=KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    score_list.append(knn.score(X_test,y_test))
    
plt.plot(range(2,15),score_list)
plt.xlabel("K values")
plt.ylabel("Accuracy")
plt.show()


* k=3 has best accuracy

In [ ]:
knn2=KNeighborsClassifier()
knn2.fit(X_train,y_train)
prediction=knn2.predict(X_test)
#Jaccard Index
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss 
js=jaccard_score(y_test,prediction,average='weighted')
print("Jaccard Index:{}".format(js))
f1=f1_score(y_test,prediction,average='weighted')
print("f1 Score:{}".format(f1))
#LogLoss for KNN=Nan



In [ ]:
#DecisionTree
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(max_depth=2)
dt.fit(X_train,y_train)
prediction2=dt.predict(X_test)
js=jaccard_score(y_test,prediction2,average='weighted')
print("Jaccard Index:{}".format(js))
f1=f1_score(y_test,prediction2,average='weighted')
print("f1 Score:{}".format(f1))
#LogLoss for DecisionTree=Nan

In [ ]:
#Visualizing DecisionTree
from sklearn import tree
tree.export_graphviz(dt)

In [ ]:
dt.predict_proba(X_train)

In [ ]:
#SVM
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn import svm

x_sc=StandardScaler()
X_train_t=x_sc.fit_transform(X_train)
X_test=x_sc.transform(X_test)

svm1=svm.SVC(kernel='linear',C=0.01)
svm1.fit(X_test,y_test)

SVC(kernel='linear',C=0.01)

y_train_pre=svm1.predict(X_train)
y_test_pre=svm1.predict(X_test)
y_test_pre
#gridsearch,c=0.1 de dene

js=jaccard_score(y_test,y_test_pre,average='weighted')
print("Jaccard Index:{}".format(js))
f1=f1_score(y_test,y_test_pre,average='weighted')
print("f1 Score:{}".format(f1))
#LogLoss for SVM=Nan

In [ ]:
### Logistic Regression
from sklearn.linear_model import LogisticRegression


lg=LogisticRegression()
lg.fit(X_train,y_train)
lg_pre_proba=lg.predict_proba(X_test)
lg_pre=lg.predict(X_test)


js=jaccard_score(y_test,lg_pre,average='weighted')
print("Jaccard Index:{}".format(js))
f1=f1_score(y_test,lg_pre,average='weighted')
print("f1 Score:{}".format(f1))
lgloss=log_loss(y_test,lg_pre_proba,normalize = True)
print("log loss:{}".format(lgloss))

### Accuracy for all
